<a href="https://colab.research.google.com/github/LeonardoRoig/TECH_5/blob/main/Vagas_pos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import ast, json, re
import warnings
warnings.filterwarnings('ignore')

# --- Nova célula ---


df_raw = pd.read_json("vagas.json")

# --- Nova célula ---

# 1) Garantir que cada célula seja um dict (pode vir como string)
def to_dict(x):
    if isinstance(x, dict):
        return x
    if pd.isna(x):
        return {}
    s = str(x)
    # tenta Python-literal
    try:
        return ast.literal_eval(s)
    except Exception:
        pass
    # tenta JSON
    try:
        return json.loads(s)
    except Exception:
        return {}

df_raw = df_raw.applymap(to_dict)

# --- Nova célula ---

# 2) Transpor: cada linha vira um job_id
df_t = df.T  # index = job_id; colunas = ['informacoes_basicas','perfil_vaga','beneficios']
df_t.index.name = "job_id"

# 3) Combinar os 3 dicionários por linha com prefixos para evitar colisão de nomes
def merge_sections(row):
    base = {f"inf_{k}": v for k, v in row.get("informacoes_basicas", {}).items()}
    perf = {f"perfil_{k}": v for k, v in row.get("perfil_vaga", {}).items()}
    benef = {f"benef_{k}": v for k, v in row.get("beneficios", {}).items()}
    return {**base, **perf, **benef}

# --- Nova célula ---

records = df_t.apply(merge_sections, axis=1)
df_flat = pd.DataFrame(list(records), index=df_t.index).reset_index()

# 4) Limpeza básica de colunas
df_flat.columns = (
    pd.Series(df_flat.columns)
    .astype(str)
    .str.strip()
    .str.lower()
    .str.normalize('NFKD')  # remove acentos
    .str.encode('ascii', errors='ignore')
    .str.decode('utf-8')
    .str.replace(r'\s+', '_', regex=True)
)

# --- Nova célula ---

# 5) Tipagem: datas (dia primeiro), valores monetários, inteiros
# Datas: qualquer coluna que contenha 'data'
for c in [c for c in df_flat.columns if 'data' in c]:
    df_flat[c] = pd.to_datetime(df_flat[c], dayfirst=True, errors='coerce')

# Função para converter dinheiro/bruto em float
money_like_cols = [c for c in df_flat.columns if any(tok in c for tok in ['valor', 'sal', 'remun', 'preco', 'custo'])]
def money_to_float(v):
    if pd.isna(v):
        return np.nan
    s = str(v).strip()
    if s in {'-', ''}:
        return np.nan
    # remove R$, espaços e separadores de milhar; troca vírgula por ponto
    s = s.replace('R$', '').replace('r$', '').replace(' ', '').replace('.', '').replace(',', '.')
    # pega o primeiro número válido
    m = re.search(r'-?\d+(\.\d+)?', s)
    return float(m.group(0)) if m else np.nan

for c in money_like_cols:
    df_flat[c] = df_flat[c].map(money_to_float)

# --- Nova célula ---

# Inteiros evidentes (ex: limites, cargas, etc.)
int_like_cols = [c for c in df_flat.columns if any(tok in c for tok in ['limite', 'carga', 'qtd', 'quant', 'horas'])]
for c in int_like_cols:
    # só converte se não for já numérico
    if not pd.api.types.is_integer_dtype(df_flat[c]):
        df_flat[c] = pd.to_numeric(df_flat[c], errors='coerce').astype('Int64')

# 6) String hygiene
str_cols = df_flat.select_dtypes(include=['object']).columns
df_flat[str_cols] = df_flat[str_cols].apply(lambda s: s.astype(str).str.strip().replace({'nan': np.nan}))

# 7) Remover colunas totalmente nulas e duplicadas
df_flat = df_flat.loc[:, df_flat.notna().any(axis=0)]
df_flat = df_flat.loc[:, ~df_flat.columns.duplicated()]

# 8) Ordenar colunas (id primeiro)
cols = ['job_id'] + [c for c in df_flat.columns if c != 'job_id']
df_final = df_flat[cols]

# --- Nova célula ---

df_final.head()

# --- Nova célula ---

df_final.isnull().sum()

# --- Nova célula ---

df_final["inf_telefone"].unique()

# --- Nova célula ---

pd.set_option('display.max_columns', None)
df_final.tail(5)

# --- Nova célula ---

vagas = df_final[["job_id", "inf_data_requicisao", "inf_titulo_vaga", "inf_vaga_sap", "inf_cliente", "inf_solicitante_cliente", "inf_empresa_divisao", "inf_requisitante","inf_analista_responsavel", "inf_tipo_contratacao",
                 "inf_prazo_contratacao", "inf_prioridade_vaga", "inf_superior_imediato", "perfil_estado", "perfil_cidade", "perfil_vaga_especifica_para_pcd", "perfil_nivel_profissional", "perfil_nivel_academico",
                 "perfil_nivel_ingles", "perfil_nivel_espanhol", "perfil_areas_atuacao", "perfil_principais_atividades", "perfil_competencia_tecnicas_e_comportamentais", "perfil_demais_observacoes", "perfil_viagens_requeridas",
                 "perfil_equipamentos_necessarios", "benef_valor_venda"]]

# --- Nova célula ---

vagas.head()

# --- Nova célula ---

vagas["inf_superior_imediato"] = vagas["inf_superior_imediato"].fillna('N/A')
vagas["perfil_equipamentos_necessarios"] = vagas["perfil_equipamentos_necessarios"].fillna('Nao Necessario')
vagas["benef_valor_venda"] = vagas["benef_valor_venda"].fillna(0)
vagas["inf_prazo_contratacao"] = vagas["inf_prazo_contratacao"].fillna('N/A')
vagas["inf_prioridade_vaga"] = vagas["inf_prioridade_vaga"].fillna('N/A')

vagas["inf_superior_imediato"] = vagas["inf_superior_imediato"].replace({'': 'N/A', 'nan': 'N/A'})
vagas["perfil_equipamentos_necessarios"] = vagas["perfil_equipamentos_necessarios"].replace({'-': 'Nao Necessario', 'nan': 'Nao Necessario'})
vagas["inf_prazo_contratacao"] = vagas["inf_prazo_contratacao"].replace({'': 'N/A', 'nan': 'N/A'})
vagas["inf_prioridade_vaga"] = vagas["inf_prioridade_vaga"].replace({'': 'N/A', 'nan': 'N/A'})

# --- Nova célula ---